In [ ]:
import re
from PyPDF2 import PdfReader
import unicodedata
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFDirectoryLoader
import os
from get_embedding_function import get_embedding_function
from langchain_chroma import Chroma
pdf_path = "/home/leonardo/Documents/python recap/guideline-170-en.pdf"
reader = PdfReader(pdf_path)
def remove_punct(text):
    return re.sub(r'[^\w\s]','', text)

def to_lower(text):
    return text.lower()

from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))

def remove_stopwords(text):
    return " ".join([word for word in text.split() if word not in stop_words])


/home/leonardo/miniconda3/envs/rag_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/leonardo/miniconda3/envs/rag_env/lib/python3.10/site-packages/torch/cuda/__init__.py:184: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:119.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:

text = ""
for page in reader.pages:
    text += page.extract_text()
    text_1 = unicodedata.normalize("NFKD", text)
    text_1 = re.sub(r'Exported on: .*', '', text_1)
    text_1 = remove_punct(text_1)
    
    text_2 = to_lower(text_1)
    text_3 = remove_stopwords(text_2)
    
with open("clean_text", 'w', encoding='unicode-escape') as f:
    f.write(text_3)

In [4]:

DATA_PATH = "data"



def load_documents():
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    return document_loader.load()

docs = load_documents()
print(docs[0])
def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)

chunked_docs = split_documents(docs)




page_content='Page 1 / 409
Exported on: 30/01/2025
Clinical guidelines - Diagnosis and treatmentmanual
For curative programmes in hospitals and dispensariesGuidance for prescribing
 
 
© Médecins Sans Frontières
All rights reserved for all countries. No reproduction, translation and adaptation may be done without the prior
permission of the Copyright owner.
We're working to ﬁx some technical issues in the MSF Guidelines app affecting ofﬂine access ×' metadata={'producer': 'Skia/PDF m91', 'creator': 'Chromium', 'creationdate': '2025-01-30T11:41:24+00:00', 'moddate': '2025-01-30T11:41:24+00:00', 'source': 'data/guideline-170-en.pdf', 'total_pages': 409, 'page': 0, 'page_label': '1'}


In [ ]:




def add_to_chroma(chunks: list[Document]):
    db = Chroma(
        persist_directory=CHROMA_PATH, embedding_function=get_embedding_function()
    )
    db.add_documents(new_chunks, ids=new_chunk_ids)
    db.persist()

In [6]:
last_page_id = None
current_chunk_index = 0
for chunk in chunked_docs:
    source = chunk.metadata.get("source")
    page = chunk.metadata.get("page")
    current_page_id = f"{source}:{page}"
    if current_page_id == last_page_id:
        current_chunk_index += 1
    else:
        current_chunk_index = 0

    chunk_id = f"{current_page_id}:{current_chunk_index}"
    last_page_id = current_page_id
    chunk.metadata["id"] = chunk_id



In [7]:
print(chunked_docs)

[Document(metadata={'producer': 'Skia/PDF m91', 'creator': 'Chromium', 'creationdate': '2025-01-30T11:41:24+00:00', 'moddate': '2025-01-30T11:41:24+00:00', 'source': 'data/guideline-170-en.pdf', 'total_pages': 409, 'page': 0, 'page_label': '1', 'id': 'data/guideline-170-en.pdf:0:0'}, page_content="Page 1 / 409\nExported on: 30/01/2025\nClinical guidelines - Diagnosis and treatmentmanual\nFor curative programmes in hospitals and dispensariesGuidance for prescribing\n\xa0\n\xa0\n© Médecins Sans Frontières\nAll rights reserved for all countries. No reproduction, translation and adaptation may be done without the prior\npermission of the Copyright owner.\nWe're working to ﬁx some technical issues in the MSF Guidelines app affecting ofﬂine access ×"), Document(metadata={'producer': 'Skia/PDF m91', 'creator': 'Chromium', 'creationdate': '2025-01-30T11:41:24+00:00', 'moddate': '2025-01-30T11:41:24+00:00', 'source': 'data/guideline-170-en.pdf', 'total_pages': 409, 'page': 1, 'page_label': '2',